In [11]:
import os
import re
import string
import pandas as pd
import urllib.parse

In [2]:
# Configuración de rutas
core_data_path = "core_data"
output_path = "docs"

# Leer los archivos TSV
colaboradores_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Colaboradores.tsv")
terminos_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Términos.tsv")

In [3]:
colaboradores_df = pd.read_csv(colaboradores_tsv, sep='\t')
terminos_df = pd.read_csv(terminos_tsv, sep='\t')

In [16]:
# Función para limpiar y generar nombres de archivo seguros
def generar_nombre_archivo(termino):
    # Convertir a minúsculas, reemplazar espacios por guiones, y eliminar caracteres no permitidos
    nombre_archivo = re.sub(r'[^a-zA-Z0-9\-]', '', termino.lower().replace(' ', '-'))
    return nombre_archivo

# Crear función para convertir términos relacionados en hipervínculos
def generar_hipervinculos(terminos, base_url):
    links = []
    for termino in terminos:
        letra_inicial = termino[0].upper()
        safe_name = generar_nombre_archivo(termino)
        link = f"[{termino}]({base_url}{letra_inicial}/{safe_name}/)"
        links.append(link)
    return ', '.join(links)

# Inicializar contadores de nav_order para cada carpeta
nav_order_alfabetico = {}
nav_order_taxonomia = {}

# Crear directorios y archivos Markdown
for _, row in terminos_df.iterrows():
    termino = row['Término']
    definicion = row['Definición']
    taxonomia = row['Taxonomía']
    relacionados = row['Relacionados'].split(', ') if pd.notna(row['Relacionados']) else []

    # Ruta base para hipervínculos
    base_url = "https://maleniski.github.io/diccionario-angl-tec-mx/docs/alfabeticamente/"

    # Convertir términos relacionados en hipervínculos
    relacionados_links = generar_hipervinculos(relacionados, base_url)

    # Determinar carpeta alfabética
    letra_inicial = termino[0].upper()
    alphabet_folder = os.path.join(output_path, "alfabeticamente", letra_inicial)
    if letra_inicial not in nav_order_alfabetico:
        nav_order_alfabetico[letra_inicial] = 1
    else:
        nav_order_alfabetico[letra_inicial] += 1

    # Crear carpeta alfabética si no existe
    os.makedirs(alphabet_folder, exist_ok=True)
    
    # Determinar carpeta de taxonomía
    taxonomy_folder = os.path.join(output_path, "taxonomia", generar_nombre_archivo(taxonomia))
    if taxonomia not in nav_order_taxonomia:
        nav_order_taxonomia[taxonomia] = 1
    else:
        nav_order_taxonomia[taxonomia] += 1

    # Crear carpeta de taxonomía si no existe
    os.makedirs(taxonomy_folder, exist_ok=True)

    # Generar nombre de archivo seguro
    md_filename = f"{generar_nombre_archivo(termino)}.md"

    # Crear contenido del archivo Markdown para la carpeta alfabética
    contenido_md_alfabetico = f"""
---
layout: default
title: {termino}
nav_order: {nav_order_alfabetico[letra_inicial]}
---

*Término:* {termino}  
*Definición:* {definicion}

*Taxonomía:* {taxonomia}  
*Términos relacionados:* {relacionados_links}
"""

    # Crear contenido del archivo Markdown para la carpeta de taxonomía
    contenido_md_taxonomia = f"""
---
layout: default
title: {termino}
nav_order: {nav_order_taxonomia[taxonomia]}
---

*Término:* {termino}  
*Definición:* {definicion}

*Taxonomía:* {taxonomia}  
*Términos relacionados:* {relacionados_links}
"""

    # Guardar archivo Markdown en carpeta alfabética
    with open(os.path.join(alphabet_folder, md_filename), 'w') as f:
        f.write(contenido_md_alfabetico)

    # Guardar archivo Markdown en carpeta de taxonomía
    with open(os.path.join(taxonomy_folder, md_filename), 'w') as f:
        f.write(contenido_md_taxonomia)